this code for compare diffrent types of feature selection techniques.
experiment 2: compare feature selection methods following the initial use of the univariate ROC-AUC method, including
two embedded techniques: LASSO and Random forest variable importance. Moreover, mutual information and ReliefF methods. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split, LeaveOneOut, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, classification_report
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier, ExtraTreesClassifier
import xgboost as xgb
from feature_engine.selection import DropDuplicateFeatures, DropConstantFeatures
from sklearn.feature_selection import SelectFromModel, VarianceThreshold
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN, BorderlineSMOTE, SVMSMOTE
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from mlxtend.classifier import EnsembleVoteClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score, plot_roc_curve
from yellowbrick.classifier import ROCAUC


# input the data (gene expression array) and labels

In [ ]:
filenames = ['riaz', 'gide', 'van allen', 'hugo', 'lee']
name= filenames[1]
name

In [ ]:
filenames = ['riaz', 'gide', 'van allen', 'hugo', 'lee']
name= filenames[1]

# Read logged data
logged_data = pd.read_csv('data/' + name + '_logged_data.csv', index_col='Unnamed: 0').T
print(logged_data.shape)
print(logged_data.head())

# Read labels
label = pd.read_csv('data/' + name + '.Pre.Samples.corr.csv', index_col='Unnamed: 0')
print(label.shape)
print(label.head())

# Count of labels
print(label['Resp_NoResp'].value_counts())

# Normalized count of labels
print(label['Resp_NoResp'].value_counts(normalize=True))

# Label encoding
y = label.replace({'No_Response': 0, 'Response': 1})
labels = ['No_Response', 'Response']  # for plotting convenience later on

print(y.head())
print(y['Resp_NoResp'].head())
print(y.iloc[:, 0].head())
print(y.iloc[:, 0].shape)


In [ ]:
#read the immune related genes from immport
IRG= pd.read_csv('IRG.csv')
print(IRG.shape)
IRG.head()

# data oversampling

In [ ]:
# Define features (X) and target (Y)
X = logged_data
Y = y.iloc[:, 0]

# Oversampling using BorderlineSMOTE
ada = BorderlineSMOTE(
    sampling_strategy='auto',  # samples only the minority class
    random_state=0,  # for reproducibility
    k_neighbors=5,
    m_neighbors=10,
    kind='borderline-1',
    n_jobs=4
)

# Resample the data
X_res, y_res = ada.fit_resample(X, Y)

# Print the shapes of resampled data
print(X_res.shape, y_res.shape)

# Print the counts of the original and resampled target variable
print(Y.value_counts(), y_res.value_counts())


# separate dataset into train and test


In [ ]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.3, random_state=0)

# Display shapes of train and test sets
print(X_train.shape, X_test.shape)

# Display first few rows of test set
print(X_test.head())

# Display normalized value counts of target variable in test set
print(y_test.value_counts(normalize=True))

# Keep a copy of the original datasets
X_train_original = X_train.copy()
X_test_original = X_test.copy()

# Check shapes of train and test sets after copying
print(X_train.shape, X_test.shape)


# use of the data selected by Univariate ROC-AUC Performance from experiment 1 to begin with experiment 2

In [ ]:
X_train = pd.read_csv('gide_X_train_uni_roc-auc.csv',index_col = 'Unnamed: 0')
print(X_train.shape)
X_train.head()


In [ ]:
X_test = pd.read_csv('gide_X_test_uni_roc-auc.csv',index_col = 'Unnamed: 0')
print(X_test.shape)
X_test.head()

In [ ]:
# keep acopy of the data selected by Univariate ROC-AUC Performance
X_train_uni = X_train
X_test_uni = X_test

# relieff

In [ ]:
X_train.shape, X_test.shape

In [ ]:
from skrebate import ReliefF
fs = ReliefF()
fs.fit(X_train.values, y_train.values)


In [ ]:
from sklearn.ensemble import VotingClassifier
accuracy_rate = []
clf1 = ExtraTreesClassifier()
clf2 = LogisticRegression(penalty ='l2', C=0.1, solver='liblinear', random_state=4,max_iter = 1000)
clf3 = SVC(gamma =1e-05, probability = True, decision_function_shape = 'ovo', kernel = 'linear')
clf4 = AdaBoostClassifier()
clf5= XGBClassifier()


# Will take some time
for i in range(5,300,5):
    
    indx_sort=fs.feature_importances_.argsort()[-i:][::-1]
    X_train_rf= X_train.iloc[:,indx_sort]
    X_test_rf= X_test.iloc[:,indx_sort]
    
    eclf1 = VotingClassifier(estimators=[ ('logistic', clf2),('SVM', clf3),
                                           ('XGB', clf5), 
                                          ('extratree',clf1), ('AdaBoost', clf4)
                                        ], voting='hard') 
    
   
    print(i)
    
    
    for clf in (clf1,clf2, clf3, clf4, clf5, eclf1):
        clf.fit(X_train_rf, y_train)
        y_pred = clf.predict(X_test_rf)
        print(clf.__class__.__name__, round(accuracy_score(y_test, y_pred),3))
        
  
    # Number of iterations
    num_iterations = 3

    # Initialize a list to store accuracy scores
    ensemble_scores = []

    # Perform multiple iterations of the VotingClassifier
    for _ in range(num_iterations):
        eclf1.fit(X_train_rf, y_train)
        predictions = eclf1.predict(X_test_rf)
        ensemble_acc = accuracy_score(y_test, predictions)
        ensemble_scores.append(ensemble_acc)

    print("Ensemble Voting Accuracy:", ensemble_scores)
    
    # Calculate the mean accuracy of the ensemble
    mean_ensemble_accuracy = np.mean(ensemble_scores)
    print("Mean Ensemble Voting Accuracy:", mean_ensemble_accuracy)

    print(X_train_rf.columns)
    accuracy_rate.append(mean_ensemble_accuracy)
    

In [ ]:
accuracy_rate

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(range(5,300,5),accuracy_rate,color='blue', linestyle='solid', marker='o',
         markerfacecolor='blue', markersize=10)

title =plt.title('ReliefF Accuracy vs. No. of Genes', fontsize=20)
plt.xlabel('No. of Genes', fontsize=16)
plt.ylabel('ReliefF Accuracy', fontsize=16)


In [ ]:
# select top 95 features ranked by Relieff, as obtainted in the increment feature selection above

In [ ]:
indx_sort=fs.feature_importances_.argsort()[-95:][::-1]
indx_sort

In [ ]:
X_train_rf= X_train.iloc[:,indx_sort]
X_train_rf.shape

In [ ]:
X_test_rf= X_test.iloc[:,indx_sort]
X_test_rf.shape

In [ ]:
X_train_rf.columns

In [ ]:
IRG_feat=X_train_rf.columns.intersection(IRG['Symbol'])
IRG_feat

In [ ]:
X_train = X_train_rf
X_test= X_test_rf


In [ ]:
#save a copy of reliefF result for the following experiment 
X_train.to_csv('gide_X_train_uni_reliefF.csv')
X_test.to_csv('gide_X_test_uni_reliefF.csv')

# Mutual information

In [ ]:
# use of the data selected by Univariate ROC-AUC Performance
X_train = X_train_uni
X_test = X_test_uni

In [ ]:
# ensure the sizes of X_train and X_test.
X_train.shape, X_test.shape

In [ ]:
# to obtain the mutual information values
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression

# to select the features
from sklearn.feature_selection import SelectKBest, SelectPercentile

In [ ]:
from sklearn.ensemble import VotingClassifier
accuracy_rate = []
clf1 = ExtraTreesClassifier()
clf2 = LogisticRegression(penalty ='l2', C=0.1, solver='liblinear', random_state=4,max_iter = 1000)
clf3 = SVC(gamma =1e-05, probability = True, decision_function_shape = 'ovo', kernel = 'linear')
clf4 = AdaBoostClassifier()
clf5= XGBClassifier()

# Will take some time
for i in range(5,300,5):
    
    sel_ = SelectKBest(mutual_info_classif, k=i).fit(X_train, y_train)
    # display features
    best_feature=X_train.columns[sel_.get_support()]
    X_train_m = sel_.transform(X_train)
    X_test_m = sel_.transform(X_test)
    X_train_m = pd.DataFrame(X_train_m,index= X_train_original.index, columns=best_feature)
    X_test_m = pd.DataFrame(X_test_m,index= X_test_original.index, columns=best_feature)
   
    
    eclf1 = VotingClassifier(estimators=[ ('logistic', clf2),('SVM', clf3),
                                           ('XGB', clf5), 
                                          ('extratree',clf1), ('AdaBoost', clf4)
                                        ], voting='hard')    
    print(i)
    
    for clf in (clf1,clf2, clf3, clf4, clf5, eclf1):
        clf.fit(X_train_m, y_train)
        y_pred = clf.predict(X_test_m)
        #df_test[clf.__class__.__name__] = y_pred
        print(clf.__class__.__name__, round(accuracy_score(y_test, y_pred),3))
    
    # Number of iterations
    num_iterations = 3

    # Initialize a list to store accuracy scores
    ensemble_scores = []

    # Perform multiple iterations of the VotingClassifier
    for _ in range(num_iterations):
        eclf1.fit(X_train_m, y_train)
        predictions = eclf1.predict(X_test_m)
        ensemble_acc = accuracy_score(y_test, predictions)
        ensemble_scores.append(ensemble_acc)

    print("Ensemble Voting Accuracy:", ensemble_scores)
    
    # Calculate the mean accuracy of the ensemble
    mean_ensemble_accuracy = np.mean(ensemble_scores)
    print("Mean Ensemble Voting Accuracy:", mean_ensemble_accuracy)

    print(X_train_m.columns)
    accuracy_rate.append(mean_ensemble_accuracy)

In [ ]:
accuracy_rate

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(range(5,300,5),accuracy_rate,color='blue', linestyle='dashed', marker='o',
         markerfacecolor='red', markersize=10)
plt.title('Mutual Info Accuracy vs. No. of Genes')
plt.xlabel('No. of Genes')
plt.ylabel('Mutual Info')

# lasso

In [ ]:
# use of the data selected by Univariate ROC-AUC Performance
X_train = X_train_uni
X_test = X_test_uni

In [ ]:
# ensure the sizes of X_train and X_test.
X_train.shape, X_test.shape

In [ ]:
X_train_uni.head()

In [ ]:
from sklearn.feature_selection import SelectFromModel
scaler = StandardScaler()
scaler.fit(X_train)

In [ ]:

sel_ = SelectFromModel(
    LogisticRegression(C=0.9, penalty='l1', solver='liblinear', random_state=0))

sel_.fit((X_train), y_train)

In [ ]:
# the number of features which coefficient was shrank to zero:
np.sum(sel_.estimator_.coef_ == 0)

In [ ]:
# we can then remove the features from the training and testing set
# like this:

X_train_lasso = sel_.transform(X_train)
X_test_lasso = sel_.transform(X_test)

X_train_lasso.shape, X_test_lasso.shape

In [ ]:
feat_names_lasso = X_train.columns[sel_.get_support()]
feat_names_lasso

In [ ]:
X_train_lasso = pd.DataFrame(X_train_lasso,index= X_train_uni.index, columns=feat_names_lasso)
X_train_lasso.head()

In [ ]:
X_test_lasso = pd.DataFrame(X_test_lasso,index= X_test_uni.index, columns=feat_names_lasso)
X_test_lasso.head()

# lasso performance evaluation

In [ ]:

clf1 = ExtraTreesClassifier()
clf2 = LogisticRegression(penalty ='l2', C=0.1, solver='liblinear', random_state=4,max_iter = 1000)
clf3 = SVC(gamma =1e-05, probability = True, decision_function_shape = 'ovo', kernel = 'linear')
clf4 = AdaBoostClassifier()
clf5= XGBClassifier()            
               
eclf1 = VotingClassifier(estimators=[  ('logistic', clf2),('SVM', clf3),
                                        ('XGB', clf5), ('extratree',clf1), 
                                        ('AdaBoost', clf4)
                                        ], voting='hard') 




for clf in (clf1,clf2, clf3, clf4, clf5, eclf1):
    clf.fit(X_train_lasso, y_train)
    y_pred = clf.predict(X_test_lasso)
    print(clf.__class__.__name__, round(accuracy_score(y_test, y_pred),3))



eclf1.fit(X_train_lasso, y_train)
predictions = eclf1.predict(X_test_lasso)
print(classification_report(y_test, predictions))

confusion_mat = confusion_matrix(y_test, predictions)
# Extract values from the confusion matrix
true_positive = confusion_mat[1, 1]
false_positive = confusion_mat[0, 1]
true_negative = confusion_mat[0, 0]
false_negative = confusion_mat[1, 0]

# Calculate Sensitivity (True Positive Rate)
sensitivity = true_positive / (true_positive + false_negative)

# Calculate Specificity
specificity = true_negative / (true_negative + false_positive)

# Print the results
print(f'Sensitivity (True Positive Rate): {sensitivity:.2f}')
print(f'Specificity: {specificity:.2f}')


# Select Features by Random Forests Importance

In [ ]:
# use of the data selected by Univariate ROC-AUC Performance
X_train = X_train_uni
X_test = X_test_uni

In [ ]:
# ensure the sizes of X_train and X_test.
X_train.shape, X_test.shape

In [ ]:
X_train_uni.head()

In [ ]:
# select features using the impotance derived from
# random forests

from sklearn.feature_selection import SelectFromModel

sel_ = SelectFromModel(RandomForestClassifier(n_estimators=10, random_state=42))
sel_.fit(X_train, y_train)



In [ ]:
# remove features with zero coefficient from dataset and parse again as dataframe (output of sklearn is numpy array)

X_train_rf = pd.DataFrame(sel_.transform(X_train),index= X_train.index, columns= X_train.columns[(sel_.get_support())])
X_test_rf = pd.DataFrame(sel_.transform(X_test),index= X_test.index, columns= X_train.columns[(sel_.get_support())])

In [ ]:
X_train_rf.shape, X_test_rf.shape

In [ ]:
X_train_rf.columns

In [ ]:
X_train = X_train_rf
X_test= X_test_rf


# RF importance performance evaluation

In [ ]:

clf1 = ExtraTreesClassifier()
clf2 = LogisticRegression(penalty ='l2', C=0.1, solver='liblinear', random_state=4,max_iter = 1000)
clf3 = SVC(gamma =1e-05, probability = True, decision_function_shape = 'ovo', kernel = 'linear')
clf4 = AdaBoostClassifier()
clf5= XGBClassifier()            
               
eclf1 = VotingClassifier(estimators=[  ('logistic', clf2),('SVM', clf3),
                                        ('XGB', clf5), ('extratree',clf1), 
                                        ('AdaBoost', clf4)
                                        ], voting='hard') 




for clf in (clf1,clf2, clf3, clf4, clf5, eclf1):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, round(accuracy_score(y_test, y_pred),3))



eclf1.fit(X_train, y_train)
predictions = eclf1.predict(X_test)
print(classification_report(y_test, predictions))

confusion_mat = confusion_matrix(y_test, predictions)
# Extract values from the confusion matrix
true_positive = confusion_mat[1, 1]
false_positive = confusion_mat[0, 1]
true_negative = confusion_mat[0, 0]
false_negative = confusion_mat[1, 0]

# Calculate Sensitivity (True Positive Rate)
sensitivity = true_positive / (true_positive + false_negative)

# Calculate Specificity
specificity = true_negative / (true_negative + false_positive)

# Print the results
print(f'Sensitivity (True Positive Rate): {sensitivity:.2f}')
print(f'Specificity: {specificity:.2f}')
